# Kernel String Encoding

Simply interpret kernels and operations as raw strings.

Example Kernel:

$K_1O_1K_2K_3O_1K_5$

Could be encoded as:

$[1, 1, 2, 3, 1, 5]$

In [171]:
import numpy as np

In [75]:
add = '+'
mult = '*'

operations = (add, mult)

kernel_families = ('A', 'B', 'C')
D = 4

kernels = [fam + str(d) for fam in kernel_families for d in range(1, D + 1)]
print('All possible kernels:')
kernels

All possible kernels:


['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4']

In [76]:
def encode_full_kernel(kernel_list, operations):
    full_kernel = np.empty((len(kernel_list) + len(operations)))
    kernel_expression = []
    for (fam, dim), op in zip(kernel_list, operations + [None]):
        kernel_expression += [encode_kernel(fam, dim)]
        if op is not None:
            kernel_expression += [encode_op(op)]

    full_kernel = np.array(kernel_expression)
    
    return full_kernel

def encode_kernel(family, dim):
    d = dim - 1
    kernel_idx = d + D * kernel_families.index(family)
    
    return kernel_idx

def encode_op(op):
    return operations.index(op)

def decode_op(op_encoded):
    return operations[op_encoded]

def decode_kernel(kernel_encoded):
    return kernels[kernel_encoded]

def decode_full_kernel(full_kernel):
    kernel = ''
    for idx, k in enumerate(full_kernel):
        if idx % 2 is 0:
            kernel += decode_kernel(k)
        else:
            kernel += decode_op(k)

    return kernel

In [78]:
# list of (kernel family, input dimension)
kernel_list = [('A', 2), ('B', 3), ('C', 1)]
operations = [add, mult]
full_kernel = encode_full_kernel(kernel_list, operations)
full_kernel

array([1, 0, 6, 1, 8])

In [77]:
decode_full_kernel(full_kernel)

'A2+B3*C1'

### Demo of Genetic Algorithm Variation

In [142]:
def crossover_one_point(parent_1, parent_2):
    '''One-point crossover
    '''
    
    L = parent_1.size
    crossover_point = np.random.randint(0, L)
    child_1 = np.hstack((parent_1[:crossover_point], parent_2[crossover_point:]))
    child_2 = np.hstack((parent_2[:crossover_point], parent_1[crossover_point:]))

    return child_1, child_2

In [154]:
# K1 = A2 + B3 * C1 + A1
# K2 = B1 * C1 * A3
kernel_list_1 = [('A', 2), ('B', 3), ('C', 1), ('A', 1)]
kernel_list_2 = [('B', 1), ('C', 1), ('A', 3)]
operations_1 = [add, mult, add]
operations_2 = [mult, mult]
full_kernel_1 = encode_full_kernel(kernel_list_1, operations_1)
full_kernel_2 = encode_full_kernel(kernel_list_2, operations_2)
full_kernel_1, full_kernel_2

(array([1, 0, 6, 1, 8, 0, 0]), array([4, 1, 8, 1, 2]))

In [146]:
child_1, child_2 = crossover_one_point(full_kernel_1, full_kernel_2)
child_1, child_2

(array([1, 0, 6, 1, 8]), array([4, 1, 8, 1, 2, 0, 0]))

In [147]:
print(('K1 = %s \nK2 = %s') % (decode_full_kernel(child_1), decode_full_kernel(child_2)))

K1 = A2+B3*C1 
K2 = B1*C1*A3+A1


### Demo of Mutation

In [148]:
def mutate_interchange(individual):
    '''Interchange mutation
    '''
    indiv_mut = individual.copy()
    L = indiv_mut.size
    ind = np.random.randint(0, L, size=2)
    # swap first and second genes
    indiv_mut = swap(indiv_mut, ind[0], ind[1])

    return indiv_mut

def swap(arr, idx_1, idx_2):
    '''Swap two array elements
    '''
    arr_copy = arr.copy()
    first = arr_copy[idx_1]
    second = arr_copy[idx_2]
    arr_copy[idx_1] = second
    arr_copy[idx_2] = first
    
    return arr_copy

In [155]:
# we are going to mutute the following kernel
# K1 = A2 + B3 * C1 + A1
full_kernel_1

array([1, 0, 6, 1, 8, 0, 0])

In [170]:
ops = full_kernel_1[1::2]
kerns = full_kernel_1[::2]

# mutate operations and kernels seperately
# make sure to reduce the mutation probability
# by 1/2
new_kernel = np.empty((len(ops) + len(kerns)), dtype=full_kernel_1.dtype)
new_kernel[1::2] = mutate_interchange(ops)
new_kernel[::2]  = mutate_interchange(kerns)

# which can be decoded as
print('K_old = %s \nK_new = %s' % (decode_full_kernel(full_kernel_1), decode_full_kernel(new_kernel)))

K_old = A2+B3*C1+A1 
K_new = A2+B3+A1*C1
